ЛДА на всем датасете

In [ ]:
#pip install --upgrade ipykernel

In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt

c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\cupy\__init__.py:103: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  from numpy import bool8  # NOQA
c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\cupy\__init__.py:103: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  from numpy import bool8  # NOQA
c:\users\philipp\appdata\local\programs\python\python39\lib\site-packages\cupy\__init__.py:114: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  from numpy import int0  # NOQA
c:\user

In [ ]:
train = pd.read_csv('train2.csv', sep='\t', encoding='ISO-8859-1')
test = pd.read_csv('test2.csv',sep='\t',encoding='ISO-8859-1')

In [ ]:
train.drop(columns = ['Unnamed: 0','body'], inplace = True)
test.drop(columns = ['Unnamed: 0','body'], inplace = True)

In [ ]:
lda_train = train.drop_duplicates(subset = ['body_new'])

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
def bigrams(words, bi_min=5, tri_min=5):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [ ]:
def get_corpus(df):

    words = list(sent_to_words(df.body_new))
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=20, no_above=0.3)
    id2word.compactify()
    corpus = [id2word.doc2bow(body_new) for body_new in bigram]
    return corpus, id2word, bigram

In [ ]:
train_corpus, train_id2word, bigram_train = get_corpus(lda_train)

In [ ]:
TOPICS = 3

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=TOPICS,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train3.model')

In [ ]:
TOPICS = 20

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=TOPICS,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train20.model')

In [ ]:
cols = train.drop(columns = ['Y', 'body_new']).columns

In [ ]:
cols = np.array(cols)

In [ ]:
def get_bigram(df):
    words = list(sent_to_words(df.body_new))
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    return bigram

In [ ]:
lda_train3 = gensim.models.ldamulticore.LdaMulticore.load('lda_train3.model')
lda_train20 = gensim.models.ldamulticore.LdaMulticore.load('lda_train20.model')

In [ ]:
train_corpus1 = [doc for doc in train_corpus if len(doc)>1]
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(lda_train3, train_corpus1, train_id2word)
p

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.320519 -0.122624       1        1  53.979800
2     -0.275717 -0.193998       2        1  26.415613
0     -0.044802  0.316622       3        1  19.604587, topic_info=           Term          Freq         Total Category  logprob  loglift
40   www_reddit  17985.000000  17985.000000  Default  30.0000  30.0000
60        imgur   8795.000000   8795.000000  Default  29.0000  29.0000
1       comment  23472.000000  23472.000000  Default  28.0000  28.0000
95         http   8451.000000   8451.000000  Default  27.0000  27.0000
0           bot   5770.000000   5770.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
38         post   1395.824194   5708.501394   Topic3  -4.8267   0.2209
365         www   1169.184697   4204.266022   Topic3  -5.0039   0.3496
144      please    996.664732   2247.547240   Topic3  -5.1635   0.8162
95         http   1253.009446   8451.206159   Topic3  -4.9346  -0.2794
246        user    910.242433   2196.820957   Topic3  -5.2542   0.7484

[156 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
2275      3  0.998812    address_jnpg
1653      3  0.999653     archive_org
32        2  0.999130         atheism
1009      3  0.999425           black
209       1  0.999059  blocked_school
...     ...       ...             ...
2285      3  0.999172         yes_bot
444       2  0.999387           youtu
366       2  0.999446         youtube
456       2  0.999671  youtube_mirror
445       2  0.998961          yt_bot

[147 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [ ]:
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(lda_train20, train_corpus1, train_id2word)
p

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
19     0.210487 -0.179182       1        1  9.426946
11    -0.211817  0.018963       2        1  7.565884
12     0.270182 -0.146696       3        1  5.783738
4      0.177670  0.061535       4        1  5.478645
18     0.003088  0.200113       5        1  5.320564
13    -0.044514 -0.224734       6        1  5.148769
7      0.093431  0.128885       7        1  5.076432
3     -0.065745  0.144462       8        1  5.036982
10    -0.065952  0.093900       9        1  4.704976
1      0.146862  0.040333      10        1  4.653317
16    -0.104090 -0.050905      11        1  4.651854
17     0.130202  0.107943      12        1  4.591150
6     -0.107145 -0.172757      13        1  4.337490
9      0.018781  0.025020      14        1  4.308072
14    -0.205796 -0.197502      15        1  4.234776
0     -0.051961  0.025684      16        1  4.090235
5     -0.024538  0.151211      17        1  4.004975
15     0.098265 -0.067386      18        1  3.951793
8     -0.093194 -0.032229      19        1  3.916405
2     -0.174217  0.073342      20        1  3.716997, topic_info=                    Term          Freq         Total Category  logprob   
40            www_reddit  19718.000000  19718.000000  Default  30.0000  \
1                comment  26207.000000  26207.000000  Default  29.0000   
347               reddit   8319.000000   8319.000000  Default  28.0000   
60                 imgur   9579.000000   9579.000000  Default  27.0000   
95                  http   9082.000000   9082.000000  Default  26.0000   
..                   ...           ...           ...      ...      ...   
875  statistic_generated    414.698589    867.532440  Topic20  -4.3776   
876     subreddits_chart    414.698589    867.532440  Topic20  -4.3776   
60                 imgur   1133.089625   9579.949404  Topic20  -3.3724   
74                 title    425.512543   3021.741272  Topic20  -4.3518   
452      message_compose    277.970043   3429.479822  Topic20  -4.7776   

     loglift  
40   30.0000  
1    29.0000  
347  28.0000  
60   27.0000  
95   26.0000  
..       ...  
875   2.5542  
876   2.5542  
60    1.1575  
74    1.3320  
452   0.7796  

[854 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
165      13  0.997753                able
878       2  0.998100           able_post
862      19  0.997447             account
284      18  0.996669  acknowledge_script
1420     13  0.995587    action_performed
...     ...       ...                 ...
366      11  0.281731             youtube
366      15  0.717854             youtube
456       1  0.382213      youtube_mirror
456      15  0.616868      youtube_mirror
445       2  0.999325              yt_bot

[968 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 12, 13, 5, 19, 14, 8, 4, 11, 2, 17, 18, 7, 10, 15, 1, 6, 16, 9, 3])

In [ ]:
bigram_train = get_bigram(train)
train_corpus = [train_id2word.doc2bow(body_new) for body_new in bigram_train]

train_vecs3 = []
for i in range(len(train)):
    top_topics = lda_train3.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(3)]
    train_vecs3.append(topic_vec)

X3 = np.array(train_vecs3)

train_vecs20 = []
for i in range(len(train)):
    top_topics = lda_train20.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    train_vecs20.append(topic_vec)

X20 = np.array(train_vecs20)

train_vecs = []

for i in range(len(train)):
  topic_vec = []
  for c in cols:
    topic_vec.extend([train.iloc[i][c]])   
  train_vecs.append(topic_vec)

features = np.array(train_vecs)
X_train = np.concatenate((X3,X20,features), axis=1)
y_train = np.array(train.Y)

In [ ]:
bigram_test = get_bigram(test)
test_corpus = [train_id2word.doc2bow(body_new) for body_new in bigram_test]

test_vecs3 = []
for i in range(len(test)):
    top_topics = lda_train3.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(3)]
    test_vecs3.append(topic_vec)

X3 = np.array(test_vecs3)

test_vecs20 = []
for i in range(len(test)):
    top_topics = lda_train20.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    test_vecs20.append(topic_vec)

X20 = np.array(test_vecs20)

test_vecs = []

for i in range(len(test)):
  topic_vec = []
  for c in cols:
    topic_vec.extend([test.iloc[i][c]])   
  test_vecs.append(topic_vec)

features = np.array(test_vecs)
X_test = np.concatenate((X3,X20,features), axis=1)
y_test = np.array(test.Y)

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, log_loss, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def metrics(y_test, y_pred, y_pred_proba):
  
  conf = confusion_matrix(y_test, y_pred)
  print(conf)
  f1_ma = round(f1_score(y_test, y_pred, average='macro'), 4)
  f1_mi = round(f1_score(y_test, y_pred, average='micro'), 4)
  f1_we = round(f1_score(y_test, y_pred, average='weighted'), 4)
  b_acc = round(balanced_accuracy_score(y_test, y_pred), 4)
  neg_ll = round(- log_loss(y_test, y_pred_proba), 4)
  roc_auc_ovr = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class = 'ovr'), 4)
  roc_auc_ovo = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class = 'ovo'), 4)

  print('f1_score_macro:', f1_ma)
  print('f1_score_micro:', f1_mi)
  print('f1_score_weighted:', f1_we)
  print('balanced_accuracy_score:', b_acc)
  print('neg_log_loss:', neg_ll)
  print('roc_auc_score ovr:', roc_auc_ovr)
  print('roc_auc_score ovo:', roc_auc_ovo)

  return pd.DataFrame({'features': ['+LDA'],'f1_score_macro': [f1_ma], 'f1_score_micro': [f1_mi],'f1_score_weighted': [f1_we], 'balanced_accuracy_score': [b_acc], 'neg_log_loss': [neg_ll],'roc_auc_score ovr': [roc_auc_ovr], 'roc_auc_score ovo': [roc_auc_ovo]})


In [ ]:
pipel = Pipeline(steps = [('scaler', StandardScaler()), ('model',LogisticRegression(max_iter=10000,class_weight = 'balanced'))])

pipel.fit(X_train, y_train)

y_pred = pipel.predict(X_test)
y_pred_proba = pipel.predict_proba(X_test)

res = metrics(y_test, y_pred, y_pred_proba)

[[38732  4693  3211]
 [   67  1351   534]
 [ 2164  4683  6308]]
f1_score_macro: 0.547
f1_score_micro: 0.7514
f1_score_weighted: 0.7905
balanced_accuracy_score: 0.6674
neg_log_loss: -0.7302
roc_auc_score ovr: 0.8852
roc_auc_score ovo: 0.876


In [ ]:
results = pd.read_csv('results.csv', sep='\t', encoding='ISO-8859-1')


In [ ]:
results = results[:2]
results = pd.concat([results, res])

In [ ]:
columns = np.concatenate((np.array(['a3','b3','c3','d20','e20','f20','g20','h20','i20','j20','k20','l20','m20','n20','o20','p20','q20','r20','s20','t20','u20','v20','w20']),cols), axis=0)

In [ ]:
test = pd.DataFrame(X_test, columns = columns)
test['Y'] = y_test

In [ ]:
train = pd.DataFrame(X_train, columns = columns)
train['Y'] = y_train

In [ ]:
train.to_csv('train3.csv', sep='\t')
test.to_csv('test3.csv', sep='\t')
results.to_csv('results.csv')